### **Tarea: Taylor** ###

Matemáticas y Estadística para Inteligencia Artificial

**Alumno**: Aminadab Córdova Acosta


Instrucciones

Los datos en la tabla muestran la probabilidad de que una persona, con un salario x, compre un producto determinado. Una función 
que comunmente se utiliza para modelar este comportamiento es la función logística:

$$ f(x) = \frac{1}{1 + e^{-(x - x_0)}} $$


donde $ x_0 $ es el punto donde la probabilidad es 0.5. Sin embargo, en términos de evaluación, es más sencillo evaluar una aproximación polinómica que evaluar esta función. Es por ello que emplearemos un polinomio de Taylor para describir estos datos.

1. Para los datos que se muestran en la tabla diga cuál cree que es el valor de $ x_0 $
2. Encuentre un polinomio de Taylor de orden 3 para la función logística alrededor de $ x_0 $
3. Evalúe el polinomio obtenido para cada uno de los salarios de la tabla
4. Encuentre el error de aproximación calculando el valor absoluto de la diferencia de los valores
aproximados y los reales
5. Grafique los valores aproximados y los reales y analice el resultado